# PoC 0
Concept: Do a super first step.

Just identify what the mean and the stdev of some data is

In [1]:
import numpy as np
import pyro
import pyro.distributions as dist
import pyro.optim as optim
import torch
import os
import matplotlib.pyplot as plt
import pyro.distributions.constraints as constraints
import logging

%matplotlib inline
plt.style.use('default')

logging.basicConfig(format='%(message)s', level=logging.INFO)
smoke_test = ('CI' in os.environ)

c:\Users\fuerf\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [51]:
mu = 1
sig = 2
n_samples = 10_000

In [55]:
data_np = np.random.normal(mu, sig, n_samples)
data = torch.tensor(data_np)



In [56]:
pyro.clear_param_store()
def model(data):
    sigma = pyro.param("sigma", torch.tensor([1.]), constraint=constraints.positive)
    mu = pyro.param("mu", torch.tensor([0.]))

    with pyro.plate("N", len(data)):
        return pyro.sample("obs", dist.Normal(mu, sigma), obs=data)

In [57]:
from pyro.infer.autoguide import AutoMultivariateNormal, init_to_mean
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [58]:
def get_svi(model):
    guide = AutoMultivariateNormal(model, init_loc_fn=init_to_mean)
    svi = SVI(model,
            guide,
            optim.Adam({"lr": .01}),
            loss=Trace_ELBO())
    return guide, svi

In [59]:
def custom_guide(data=None):
    sigma = pyro.param("sigma", lambda: torch.tensor([1.]), constraint=constraints.positive)
    mu = pyro.param("mu", lambda: torch.tensor([0.]))

    return {"mu": mu, "sigma": sigma}

In [60]:
# setup the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, custom_guide, optimizer, loss=Trace_ELBO())

In [61]:
n_steps = 10000
# do gradient steps
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print('.', end='')

....................................................................................................

In [62]:
# grab the learned variational parameters
mu = pyro.param("mu").item()
sigma = pyro.param("sigma").item()

print(f"bayesian: mu={mu}, sigma={sigma}")
print(f"Check: mu={np.mean(data_np)}, sigma={np.std(data_np)}")

bayesian: mu=0.9809330701828003, sigma=1.9738434553146362
Check: mu=0.9809344359584715, sigma=1.9738422935781812


# Next Level

In [12]:
def gen_data(gen_avg, gen_sig, n_samples, contributer_coefficients):
    """generate random data.
    
    Inputs:
    * gen_avg: mean for the baseline
    * gen_sig: sigma for the baseline
    * n_samples: number of samples
    * contributer_coefficients: dictionary with (mean,sig) per level per contributer
    
    Outputs:
    * data: array with the final number
    * contributers: matrix defining the contributers, first column is for the first level"""
    
    data = np.random.normal(gen_avg, gen_sig, n_samples)
    contributers = np.zeros((n_samples, len(contributer_coefficients)))
    for lvl, cdict in contributer_coefficients.items():
        print(f"creating level {lvl}")
        lvl_influencers = len(cdict) #number of influencers in this level
        lvldata = np.zeros((n_samples, lvl_influencers))

        for i, (mu,sig) in cdict.items():
            lvldata[:,i] = np.random.normal(mu, sig, n_samples)

        selection = np.random.randint(low=0,
                                    high=lvl_influencers,
                                    size=(n_samples))
        contributers[:, lvl] = selection
        
        data += np.array([lvldata[row, col] for row, col in enumerate(selection)])
    return torch.from_numpy(data), torch.from_numpy(contributers)

In [13]:
cc_01 = {0: {    0: (1, 1),
                1: (-1, 1)}}

In [14]:
# generate the data
data, contributers = gen_data(gen_avg=mu,
                              gen_sig=sig,
                              n_samples=10000,
                              contributer_coefficients=cc_01)

creating level 0


In [18]:
np.std(data.numpy())

2.441356683749619

In [24]:
np.mean(data.numpy())

0.9646276998710626

In [31]:
contributers[:,0].shape

torch.Size([10000])

In [35]:
np.mean(data[contributers[:,0]==1].numpy())

0.016370708156036733

In [38]:
list(contributers.unique().numpy())

[0.0, 1.0]

In [43]:
def test(data, contributers):
    n_contributers = contributers.shape[1]
    for step in range(n_contributers):
        cons = list(contributers[:,step].unique().numpy())
        for c in cons:
            d = data[contributers[:,step]==c].numpy()
            print(f"step{step}, con {c} : mu={np.mean(d)}, sig={np.std(d)**2}")

In [44]:
test(data, contributers)

step0, con 0.0 : mu=1.934366241560215, sig=5.0991247731594145
step0, con 1.0 : mu=0.016370708156036733, sig=4.983492518090686


In [82]:
pyro.clear_param_store()
def custom_guide2(data=None):
    sigma = pyro.param("sigma", lambda: torch.tensor([1.]), constraint=constraints.positive)
    mu = pyro.param("mu", lambda: torch.tensor([0.]))

    return {"mu": mu, "sigma": sigma}

def model2(data, contributers):
    """
    Concept: means are added, variances are summed --> works
    """
    #sigma = pyro.param("sigma", torch.tensor([1.]), constraint=constraints.interval(0.01,5))
    # idea: define the parameter for the std deviation as a ratio
    # sigma = 0.5 --> the other two will be another 0.5 and the first one will define his part.
    mu = pyro.param("mu", torch.tensor([0.]))
    sigma_measured = 5

    #a_s = pyro.param("a_s", dist.LogNormal(4, 3.))

    b_0_m = pyro.param("b_0_m", dist.Normal(0., 1.))

    sigma = pyro.param("b_0_s", dist.LogNormal(0., 1.), constraint=constraints.positive)
    b_0_sigma = pyro.param("b_0_sigma", dist.LogNormal(0., 1.), constraint=constraints.positive)
    sigma_np = sigma.detach().numpy()
    s = np.sqrt(sigma_measured * sigma_np)
    b_0_sigma_np = b_0_sigma.detach().numpy()
    b_0_s = np.sqrt(b_0_sigma_np * (1-sigma_np) * sigma_measured)
    b_1_s = np.sqrt((1-b_0_sigma_np) * (1-sigma_np) * sigma_measured)

    with pyro.plate("N", len(data)):
        b_0 = pyro.sample("b_0", dist.Normal(b_0_m, b_0_s))
        b_1 = pyro.sample("b_1", dist.Normal(-b_0_m, b_1_s))
        #sum of b_0 and b_1 is zero (on avg)

        #sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
        mean = mu + (contributers == 0) * b_0 + (contributers == 1) * b_1
        return pyro.sample("obs", dist.Normal(mean, s), obs=data)

In [83]:
from pyro.contrib.autoguide import AutoDiagonalNormal
guide_adn = AutoDiagonalNormal(model2)

In [84]:
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model2, guide_adn, optimizer, loss=Trace_ELBO())

In [85]:
n_steps = 10000
# do gradient steps
for step in range(n_steps):
    svi.step(data, contributers)
    if step % 100 == 0:
        print('.', end='')

C:\Users\fuerf\AppData\Local\Temp\ipykernel_13780\2283872440.py:26: RuntimeWarning: invalid value encountered in sqrt
  b_0_s = np.sqrt(b_0_sigma_np * (1-sigma_np) * 5)
C:\Users\fuerf\AppData\Local\Temp\ipykernel_13780\2283872440.py:27: RuntimeWarning: invalid value encountered in sqrt
  b_1_s = np.sqrt((1-b_0_sigma_np) * (1-sigma_np) * 5)


ValueError: Expected parameter scale (Tensor of shape ()) of distribution Normal(loc: -1.8690769672393799, scale: nan) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
nan
Trace Shapes:
 Param Sites:
Sample Sites:
Trace Shapes:
 Param Sites:
Sample Sites:

In [70]:
sigma = pyro.param("b_0_s", dist.LogNormal(0., 1.), constraint=constraints.positive)
b_0_sigma = pyro.param("b_0_sigma", dist.LogNormal(0., 1.), constraint=constraints.positive)

s = np.sqrt(5 * sigma.detach().numpy())